In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import joblib
# Define the list of CSV files
csv_files = [
    'data/Youtube01-Psy.csv',
    'data/Youtube02-KatyPerry.csv',
    'data/Youtube03-LMFAO.csv',
    'data/Youtube04-Eminem.csv',
    'data/Youtube05-Shakira.csv'
]

# Load and combine all CSV files into a single DataFrame
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate the dataframes vertically
data = pd.concat(dataframes, ignore_index=True)


In [2]:
data

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
1951,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0
1952,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0
1953,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0
1954,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0


In [3]:
data.loc[4,'CONTENT']

'watch?v=vtaRGgvGtWQ   Check this out .\ufeff'

In [4]:
data.loc[1952,'CONTENT']

'I love this song for two reasons: 1.it is about Africa 2.i was born in beautiful south Africa'

In [5]:
# Text preprocessing function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
    
    # Tokenization (split text into words)
    tokens = word_tokenize(text)
    
    # Remove stopwords (common words like "the", "and", "is")
    tokens = [word for word in tokens if word not in stop_words]
    
    # Rejoin tokens into a single string
    text = ' '.join(tokens)
    
    return text
# Apply text preprocessing to the 'sms' column
data['CONTENT'] = data['CONTENT'].apply(preprocess_text)

X = vectorizer.fit_transform(data['CONTENT'].values)
y = data['CLASS'].values

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'string' is not defined

In [ ]:
# Train different classifiers: Random Forest, SVM, Gradient Boosting
random_forest = RandomForestClassifier()
svm = SVC(probability=True)  # Enable probability estimates for VotingClassifier
gradient_boosting = GradientBoostingClassifier()

random_forest.fit(X_train, y_train)
svm.fit(X_train, y_train)
gradient_boosting.fit(X_train, y_train)

# Make predictions on the test set for each model
rf_predictions = random_forest.predict(X_test)
svm_predictions = svm.predict(X_test)
gb_predictions = gradient_boosting.predict(X_test)

# Calculate evaluation metrics (Accuracy, Precision, Recall, F1-Score, Confusion Matrix) for each model
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_f1 = f1_score(y_test, rf_predictions)
rf_confusion = confusion_matrix(y_test, rf_predictions)

svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)
svm_f1 = f1_score(y_test, svm_predictions)
svm_confusion = confusion_matrix(y_test, svm_predictions)

gb_accuracy = accuracy_score(y_test, gb_predictions)
gb_precision = precision_score(y_test, gb_predictions)
gb_recall = recall_score(y_test, gb_predictions)
gb_f1 = f1_score(y_test, gb_predictions)
gb_confusion = confusion_matrix(y_test, gb_predictions)

# Print evaluation metrics for each individual model
print("Random Forest Metrics:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1-Score:", rf_f1)
print("Confusion Matrix:")
print(rf_confusion)

print("\nSVM Metrics:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("F1-Score:", svm_f1)
print("Confusion Matrix:")
print(svm_confusion)

print("\nGradient Boosting Metrics:")
print("Accuracy:", gb_accuracy)
print("Precision:", gb_precision)
print("Recall:", gb_recall)
print("F1-Score:", gb_f1)
print("Confusion Matrix:")
print(gb_confusion)

# Create an ensemble of models using a VotingClassifier with a "soft" voting strategy
ensemble = VotingClassifier(estimators=[
    ('Random Forest', random_forest),
    ('SVM', svm),
    ('Gradient Boosting', gradient_boosting)
], voting='soft')

ensemble.fit(X_train, y_train)
# Visualize performance metrics using Seaborn
metrics_data = {
    'Model': ['Random Forest', 'SVM', 'Gradient Boosting', 'Ensemble'],
    'Accuracy': [rf_accuracy, svm_accuracy, gb_accuracy, ensemble_accuracy],
    'Precision': [rf_precision, svm_precision, gb_precision, ensemble_precision],
    'Recall': [rf_recall, svm_recall, gb_recall, ensemble_recall],
    'F1-Score': [rf_f1, svm_f1, gb_f1, ensemble_f1]
}

metrics_df = pd.DataFrame(metrics_data)

plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Accuracy', data=metrics_df)
plt.title('Accuracy Comparison')
plt.ylim(0, 1)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Precision', data=metrics_df)
plt.title('Precision Comparison')
plt.ylim(0, 1)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Recall', data=metrics_df)
plt.title('Recall Comparison')
plt.ylim(0, 1)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='F1-Score', data=metrics_df)
plt.title('F1-Score Comparison')
plt.ylim(0, 1)
plt.show()

# Visualize confusion matrices using Seaborn
def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(title)

plt.figure(figsize=(16, 6))
plot_confusion_matrix(rf_confusion, ['Not Spam', 'Spam'], 'Random Forest')
plot_confusion_matrix(svm_confusion, ['Not Spam', 'Spam'], 'SVM')
plot_confusion_matrix(gb_confusion, ['Not Spam', 'Spam'], 'Gradient Boosting')
plot_confusion_matrix(ensemble_confusion, ['Not Spam', 'Spam'], 'Ensemble')

plt.tight_layout()
plt.show()